In [21]:
import os
import json
import pandas as pd
from tqdm import tqdm
import tiktoken
from dotenv import load_dotenv
import time
import warnings
warnings.filterwarnings('ignore')

# LangChain Import
from langchain.llms import OpenAI # for new models
from langchain.chat_models import ChatOpenAI # for older models
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import SystemMessage, HumanMessage

# Load OpenaAI API key
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define Classification Function

In [22]:
def count_tokens(text):
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))

def classify_sentence(input_text, template, model="gpt-4o"):
    
    # Define a prompt template for classification
    prompt_template = PromptTemplate(
        input_variables=["text"],
        template=template
    )

    # Create an OpenAI LLM instance
    #llm = OpenAI(                    # for newer models
    llm = ChatOpenAI(                 # for older models
        model=model,
        temperature=0,
        max_retries=2,
        max_tokens=10  # Slightly increased for safety
    )

    # Set up the classification chain
    classification_chain = LLMChain(llm=llm, prompt=prompt_template)

    # Perform Classification
    output_string = classification_chain.run(text=input_text).strip()

    # Calculate token count
    input_count = count_tokens(prompt_template.format(text=input_text))
    output_count = count_tokens(output_string)

    # TWEAK: add output_token and input token than modify the rest

    # Debugging statement (prints the formatted prompt)
    #print(f"Generated Prompt:\n{prompt_template.format(text=input_text)}")

    return output_string, input_count, output_count


In [23]:
# Sample Sentences for Testing
input_text = "Additionally, the stopper 108 is used at the distal end of the wire where the loop is formed to substantially secure the loop closed." # MIX
input_text = "Provisional Patent Application number 62/571,193; filed Oct. 11, 2017; and entitled INSECT VACUUM AND TRAP ATTACHMENT SYSTEMS." #OTH
input_text = "In some embodiments, the horizontal position of the idler support block 1213 may be adjustable to maintain tension on the chain 1212." #FUN
input_text = "If there are no allocated cells to a hub using the previous criterion, the first allocated cell will be the closest cell to that hub." #FUN
input_text = "The rigid foam layer 50 is typically selected from the group of polyurethane foams, polyurea foams, and combinations thereof." # STR

# Define Initial Prompt Template for testing
template = """
Your task is to classify a given sentence as either: 
* 'FUN' - if the sentence describes only the functioning or behavior of a device;
* 'STR' - if the sentence describes only the structure or architecture of a device;
* 'MIX' - if the sentence describes both the functioning and the structure of a device;
* 'OTH' - if the sentence cannot be classified according to any of the previous classes.
The output should only contain one of the class labels: 'FUN', 'STR', 'MIX' or 'OTH'.
Sentence: "{text}"
Class:
"""

# Test the Classification Function
output_string, input_count, output_count = classify_sentence(input_text=input_text, template=template)
print(f"Input Tokens: {input_count}")
print(f"Output Tokens: {output_count}")
print(f"Predicted Class: {output_string}") 

Input Tokens: 147
Output Tokens: 1
Predicted Class: STR


# Load Prompts

In [24]:
with open("prompt_templates.json", "r") as file:
        templates = json.load(file)

# Vizualize templates
for name, template in templates.items():
    print(f"{name}:")
    print(template)
    print()

prompt1:
Your task is to classify a given sentence as either: 
* 'FUN' - if the sentence describes only the functioning or behavior of a device;
* 'STR' - if the sentence describes only the structure or architecture of a device;
* 'MIX' - if the sentence describes both the functioning and the structure of a device;
* 'OTH' - if the sentence cannot be classified according to any of the previous classes.
The output should only contain one of the class labels: 'FUN', 'STR', 'MIX' or 'OTH'.
Sentence: '{text}' 
Class:

prompt2:
Patent documents describe the functioning and the structure of patented systems.
Your task is to classify a given sentence as either: 
* 'FUN' - if the sentence describes only the functioning or behavior of a device;
* 'STR' - if the sentence describes only the structure or architecture of a device;
* 'MIX' - if the sentence describes both the functioning and the structure of a device;
* 'OTH' - if the sentence cannot be classified according to any of the previous cl

#  Classify Sentences

In [ ]:
# Import data
test_df = pd.read_excel("/home/fantoni/patent-sentence-classification/data/test_agreement.xlsx")
test_df

In [26]:
# Set the model to use
model =  "gpt-3.5-turbo"
model =  "gpt-4o"

# Iterate over templates
for templ_name, template in templates.items():
    if templ_name in ['prompt1', 'prompt3','prompt5', 'prompt8']:  # choose and skip specific templates
    
        # Initialize results dictionary
        results = {
            'sent_id': [],
            'sent': [],
            'true_class': [],
            'pred_class': [],
            'model_output': [],
            'input_count': [],
            'output_count': [],
            'errors': [],
            'elapsed_time_sec': []
        }

        # Perform Classification
        for i, row in tqdm(test_df.head(1).iterrows(), total=len(test_df), desc="Processing Sentences"):  
            start_time = time.time()              
            try:
                results['sent_id'].append(row['sent_id'])
                results['sent'].append(row['sent'])
                results['true_class'].append(row['sent_tag'])

                # Get classification
                output_string, input_count, output_count = classify_sentence(input_text=row['sent'], template=template, model=model)

                # Validate classification output
                if output_string not in ["FUN", "STR", "MIX", "OTH"]:
                    raise ValueError(f"Invalid Classification Tag.")

                # Append Results 
                results['pred_class'].append(output_string)
                results['model_output'].append(output_string)
                results['errors'].append(None)  # No error occurred
                results['input_count'].append(input_count)  # Append input token count when successful
                results['output_count'].append(output_count)

            except Exception as e:
                # Handle any errors that occur during processing
                print(f"Error in sentence {row['sent_id']}: {e}. Skipping...")
                results['pred_class'].append("ERROR")
                results['model_output'].append(output_string)
                results['input_count'].append("ERROR") 
                results['output_count'].append("ERROR") 
                results['errors'].append(str(e))
            
            finally:
                # Tracking how long it took to process the sentence
                end_time = time.time()
                elapsed_time = end_time-start_time
                results['elapsed_time_sec'].append(f"{elapsed_time}")

        # Convert results to DataFrame and Save
        #result_df = pd.DataFrame(results)
        #result_df.to_excel(f'/home/fantoni/patent-sentence-classification/results/prompting/{model}_{templ_name}.xlsx', index = False)
        print(f"{templ_name} completed.")

Processing Sentences:   0%|          | 1/1200 [00:00<13:39,  1.46it/s]


prompt1 completed.


Processing Sentences:   0%|          | 1/1200 [00:00<08:58,  2.23it/s]


prompt3 completed.


Processing Sentences:   0%|          | 1/1200 [00:00<11:52,  1.68it/s]


prompt5 completed.


Processing Sentences:   0%|          | 1/1200 [00:00<09:49,  2.03it/s]

prompt8 completed.
